In [1]:
%cd /Users/shivamkaushik/Code/ik-agent

/Users/shivamkaushik/Code/ik-agent


In [2]:
import os
from imagekitio import ImageKit
from IPython.display import Image, display

In [3]:
from typing import Optional, Union, List, Dict, Any, Literal
from pydantic import BaseModel, Field, model_validator


NumberOrExpression = Union[int, float, str]


class EUSM(BaseModel):
    radius: int
    sigma: int
    amount: float
    threshold: float


class EShadow(BaseModel):
    blur: int = 10
    saturation: int = 30
    x_offset: int = 2
    y_offset: int = 2


class EGradient(BaseModel):
    linear_direction: Union[int, str] = 180
    from_color: str = "FFFFFF"
    to_color: str = "000000"
    stop_point: Union[int, str] = 1


class EDistort(BaseModel):
    x1: int
    y1: int
    x2: int
    y2: int
    x3: int
    y3: int
    x4: int
    y4: int
    type: Literal["perspective", "arc"] = "perspective"
    arc_degree: Optional[int] = None


class ImageOverlay(BaseModel):
    """
    Pydantic representation of an ImageKit image overlay.
    """

    # -------------------------------------------------
    # IMAGE SOURCE
    # -------------------------------------------------
    image_path: Optional[str] = None
    image_path_encoded: Optional[str] = None

    # -------------------------------------------------
    # SIZE & CROP
    # -------------------------------------------------
    w: Optional[NumberOrExpression] = None
    h: Optional[NumberOrExpression] = None
    ar: Optional[NumberOrExpression] = None

    c: Optional[Literal["force", "at_max", "at_least"]] = None
    cm: Optional[Literal["extract", "pad_resize"]] = None
    fo: Optional[
        Literal[
            "face",
            "center",
            "top",
            "bottom",
            "left",
            "right",
            "top_left",
            "top_right",
            "bottom_left",
            "bottom_right",
        ]
    ] = None
    z: Optional[NumberOrExpression] = None

    x: Optional[NumberOrExpression] = None
    y: Optional[NumberOrExpression] = None
    xc: Optional[NumberOrExpression] = None
    yc: Optional[NumberOrExpression] = None

    # -------------------------------------------------
    # POSITION
    # -------------------------------------------------
    lx: Optional[NumberOrExpression] = None
    ly: Optional[NumberOrExpression] = None
    lfo: Optional[str] = None

    # -------------------------------------------------
    # APPEARANCE
    # -------------------------------------------------
    bg: Optional[str] = None
    b: Optional[str] = None
    o: Optional[int] = None
    r: Optional[Union[int, str]] = None
    rt: Optional[int] = None
    fl: Optional[str] = None
    q: Optional[int] = None
    bl: Optional[int] = None
    dpr: Optional[Union[float, str]] = None
    t: Optional[Union[bool, int]] = None

    # -------------------------------------------------
    # EFFECTS
    # -------------------------------------------------
    e_grayscale: bool = False
    e_contrast: bool = False
    e_sharpen: Union[bool, int] = False
    e_usm: Union[EUSM, bool] = False
    e_shadow: Union[EShadow, bool] = False
    e_gradient: Union[EGradient, bool] = False
    e_distort: Union[EDistort, bool] = False

    enabled: bool = True

    # -------------------------------------------------
    # NESTED OVERLAY (ONLY ONE)
    # -------------------------------------------------
    child: Optional["ImageOverlay"] = None


    @model_validator(mode="after")
    def validate_image_source(self):
        if not (self.image_path or self.image_path_encoded):
            raise ValueError("ImageOverlay requires image_path or image_path_encoded")

        if self.image_path and self.image_path_encoded:
            raise ValueError("Use only one of image_path or image_path_encoded")

        if self.z is not None and self.fo != "face":
            raise ValueError("z (zoom) requires fo='face'")

        if any(v is not None for v in (self.x, self.y, self.xc, self.yc)):
            if self.cm != "extract":
                raise ValueError("x/y/xc/yc require cm='extract'")

        if self.dpr is not None and not (self.w or self.h):
            raise ValueError("dpr requires w or h")

        return self

    def to_overlay_dict(self) -> Dict[str, Any]:
        if not self.enabled:
            return {}

        overlay: Dict[str, Any] = {
            "type": "image",
            "input": self.image_path or self.image_path_encoded,
            "encoding": "auto",
            "position": {},
            "timing": {},
            "transformation": [],
        }

        # Position
        if self.lx is not None:
            overlay["position"]["x"] = self.lx
        if self.ly is not None:
            overlay["position"]["y"] = self.ly
        if self.lfo is not None:
            overlay["position"]["focus"] = self.lfo

        transform: Dict[str, Any] = {}

        for attr in (
            "w",
            "h",
            "ar",
            "c",
            "cm",
            "fo",
            "z",
            "x",
            "y",
            "xc",
            "yc",
            "bg",
            "b",
            "o",
            "r",
            "rt",
            "fl",
            "q",
            "bl",
            "dpr",
            "t",
        ):
            val = getattr(self, attr)
            if val is not None:
                transform[attr] = val

        if self.e_grayscale:
            transform["e"] = "grayscale"
        if self.e_contrast:
            transform["e"] = "contrast"

        if self.e_sharpen:
            transform["e-sharpen"] = (
                self.e_sharpen if isinstance(self.e_sharpen, int) else None
            )

        if isinstance(self.e_usm, EUSM):
            transform["e-usm"] = (
                f"{self.e_usm.radius}-{self.e_usm.sigma}-"
                f"{self.e_usm.amount}-{self.e_usm.threshold}"
            )

        if isinstance(self.e_shadow, EShadow):
            transform["e-shadow"] = (
                f"{self.e_shadow.blur}-{self.e_shadow.saturation}-"
                f"{self.e_shadow.x_offset}-{self.e_shadow.y_offset}"
            )

        if isinstance(self.e_gradient, EGradient):
            transform["e-gradient"] = (
                f"ld-{self.e_gradient.linear_direction}_"
                f"from-{self.e_gradient.from_color}_"
                f"to-{self.e_gradient.to_color}_"
                f"sp-{self.e_gradient.stop_point}"
            )

        if isinstance(self.e_distort, EDistort):
            if self.e_distort.type == "perspective":
                transform["e-distort"] = (
                    f"p-{self.e_distort.x1}_{self.e_distort.y1}_"
                    f"{self.e_distort.x2}_{self.e_distort.y2}_"
                    f"{self.e_distort.x3}_{self.e_distort.y3}_"
                    f"{self.e_distort.x4}_{self.e_distort.y4}"
                )
            else:
                transform["e-distort"] = f"a-{self.e_distort.arc_degree}"

        if transform:
            overlay["transformation"].append(transform)

        # -------------------------------------------------
        # NESTED OVERLAY (single)
        # -------------------------------------------------
        if self.child is not None:
            # child.to_overlay_dict() returns {"overlay": {...}}
            transform["overlay"] = self.child.to_overlay_dict()["overlay"]

        if transform:
            overlay["transformation"].append(transform)


        if not overlay["position"]:
            overlay.pop("position")
        if not overlay["timing"]:
            overlay.pop("timing")
        if not overlay["transformation"]:
            overlay.pop("transformation")

        return {"overlay": overlay}


In [ ]:
parent = ImageOverlay(
    image_path="/logo-white_SJwqB4Nfe.png",
    w=100,
    h=50,
    e_grayscale=True,
    lx=10,
    ly=10
)
transformation = parent.to_overlay_dict()


client = ImageKit(
    private_key=os.environ.get("IMAGEKIT_PRIVATE_KEY"),
)

src="medium_cafe_B1iTdD0C.jpg"
# URL with image overlay
url = client.helper.build_url(
    url_endpoint="https://ik.imagekit.io/demo",
    src=src,
    transformation=[
        transformation
    ],
)
display(Image(url=url))
# Result: URL with image overlay positioned at x:10, y:10

In [12]:
parent = ImageOverlay(
    image_path="/logo-white_SJwqB4Nfe.png",
    lx=100,
    ly=100,
    # e_grayscale=True
)
transformation = parent.to_overlay_dict()


client = ImageKit(
    private_key=os.environ.get("IMAGEKIT_PRIVATE_KEY"),
)

src="medium_cafe_B1iTdD0C.jpg"
# URL with image overlay
url = client.helper.build_url(
    url_endpoint="https://ik.imagekit.io/demo",
    src=src,
    transformation=[
        transformation
    ],
)

print(url)

display(Image(url=url))
# Result: URL with image overlay positioned at x:10, y:10

https://ik.imagekit.io/demo/medium_cafe_B1iTdD0C.jpg?tr=l-image,i-logo-white_SJwqB4Nfe.png,lx-100,ly-100,l-end
